In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install autogluon

In [3]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/multitude_split/dataset_all.csv')
data.head()

,text,label,multi_label,split,language,length,source,word_count,unique_word_count,char_count,...,question_mark_count,exclamation_mark_count,flesch_reading_ease,gunning_fog_index,first_person_pronoun_count,person_entity_count,date_entity_count,uniqueness_bigram,uniqueness_trigram,syntax_variety
0,Der Ausbruch des Coronavirus hat die Entwicklu...,1,text-davinci-003,test,de,174,MULTITuDE_MassiveSumm_spiegel,199.0,118.0,1067.0,...,0.0,0.0,-272.022170,11.156030,0.0,0.0,0.0,0.904040,0.979695,12.0
1,Alex Azar was officially sworn in as the U.S. ...,1,text-davinci-003,train,en,57,MULTITuDE_MassiveSumm_voanews,70.0,54.0,311.0,...,0.0,1.0,-186.793214,8.714286,0.0,5.0,2.0,1.000000,1.000000,11.0
2,Європейський союз вимагає зупинити розтрату ко...,1,gpt-3.5-turbo,test,uk,105,MULTITuDE_MassiveSumm_interfax,130.0,82.0,691.0,...,0.0,0.0,-269.236538,11.015385,0.0,0.0,0.0,0.860465,0.929688,14.0
3,"Yesterday, hundreds of Zambian university stud...",1,text-davinci-003,train,en,254,MULTITuDE_MassiveSumm_voanews,292.0,149.0,1419.0,...,0.0,0.0,-231.229869,11.440100,1.0,1.0,1.0,0.876289,0.965517,13.0
4,"In a narrow and highly watched vote, the US Se...",1,gpt-4,train,en,416,MULTITuDE_MassiveSumm_voanews,476.0,242.0,2259.0,...,0.0,0.0,-224.855788,13.160504,1.0,2.0,2.0,0.871579,0.974684,15.0


In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

statistical_features = ['word_count', 'unique_word_count', 'char_count', 'avg_word_length',
       'ttr', 'hapax_legomenon', 'sentence_count', 'avg_sentence_length',
       'avg_sentence_complexity', 'punctuation_count', 'noun_count',
       'stopword_count', 'verb_count', 'adj_count', 'adv_count',
       'complex_sentence_count', 'question_mark_count',
       'exclamation_mark_count', 'flesch_reading_ease', 'gunning_fog_index',
       'first_person_pronoun_count', 'person_entity_count',
       'date_entity_count', 'uniqueness_bigram', 'uniqueness_trigram',
       'syntax_variety']
# Assuming you have a 'split' column to distinguish between train and test data
train_data = data[data['split'] == 'train']
test_data = data[data['split'] == 'test']

# Fit and transform the training data
train_data[statistical_features] = scaler.fit_transform(train_data[statistical_features])

# Transform the test data based on the scaler fitted to the training data
test_data[statistical_features] = scaler.transform(test_data[statistical_features])

# Extract features and labels
train_data = train_data[['word_count', 'unique_word_count', 'char_count', 'avg_word_length',
       'ttr', 'hapax_legomenon', 'sentence_count', 'avg_sentence_length',
       'avg_sentence_complexity', 'punctuation_count', 'noun_count',
       'stopword_count', 'verb_count', 'adj_count', 'adv_count',
       'complex_sentence_count', 'question_mark_count',
       'exclamation_mark_count', 'flesch_reading_ease', 'gunning_fog_index',
       'first_person_pronoun_count', 'person_entity_count',
       'date_entity_count', 'uniqueness_bigram', 'uniqueness_trigram',
       'syntax_variety', 'label']]

test_data = test_data[['word_count', 'unique_word_count', 'char_count', 'avg_word_length',
       'ttr', 'hapax_legomenon', 'sentence_count', 'avg_sentence_length',
       'avg_sentence_complexity', 'punctuation_count', 'noun_count',
       'stopword_count', 'verb_count', 'adj_count', 'adv_count',
       'complex_sentence_count', 'question_mark_count',
       'exclamation_mark_count', 'flesch_reading_ease', 'gunning_fog_index',
       'first_person_pronoun_count', 'person_entity_count',
       'date_entity_count', 'uniqueness_bigram', 'uniqueness_trigram',
       'syntax_variety', 'label']]

<ipython-input-4-c4f620b7397a>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[statistical_features] = scaler.fit_transform(train_data[statistical_features])
<ipython-input-4-c4f620b7397a>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[statistical_features] = scaler.transform(test_data[statistical_features])


In [5]:
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [6]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [7]:
train_data = TabularDataset(train_data)
train_data.head()

,word_count,unique_word_count,char_count,avg_word_length,ttr,hapax_legomenon,sentence_count,avg_sentence_length,avg_sentence_complexity,punctuation_count,...,exclamation_mark_count,flesch_reading_ease,gunning_fog_index,first_person_pronoun_count,person_entity_count,date_entity_count,uniqueness_bigram,uniqueness_trigram,syntax_variety,label
0,0.125737,0.158537,0.110176,0.048981,0.768652,0.614286,0.052632,0.031683,0.067629,0.035831,...,0.017241,0.954760,0.040009,0.000000,0.121951,0.047619,1.000000,1.000000,0.600000,1
1,0.561886,0.448171,0.534047,0.055545,0.504326,0.376712,0.175439,0.049288,0.043773,0.104235,...,0.000000,0.946426,0.053268,0.009174,0.024390,0.023810,0.874273,0.964811,0.733333,1
2,0.923379,0.731707,0.855394,0.053752,0.502433,0.386555,0.263158,0.055526,0.038802,0.153094,...,0.000000,0.947622,0.061636,0.009174,0.048780,0.047619,0.869487,0.974165,0.866667,1
3,0.758350,0.609756,0.607881,0.043773,0.509419,0.369898,0.192982,0.061202,0.035125,0.182410,...,0.000000,0.957118,0.065150,0.110092,0.219512,0.047619,0.890833,0.979067,0.800000,1
4,0.045187,0.073171,0.042081,0.051239,0.895295,0.793103,0.000000,0.054066,0.039867,0.019544,...,0.000000,0.950296,0.060754,0.000000,0.000000,0.000000,1.000000,1.000000,0.600000,1


In [10]:
predictor = TabularPredictor(label='label').fit(
    train_data,
    num_gpus=1,  # Grant 1 gpu for the entire Tabular Predictor
    presets='best_quality',
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240703_125126"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Tue Jun 18 14:18:04 UTC 2024
CPU Count:          2
Memory Avail:       9.98 GB / 12.67 GB (78.8%)
Disk Space Avail:   169.61 GB / 201.23 GB (84.3%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be

In [11]:
leaderboard = predictor.leaderboard(
    test_data,
    silent=True,
    extra_metrics=['accuracy', 'roc_auc', 'f1_macro', 'f1_weighted']
)
leaderboard

,model,score_test,accuracy,roc_auc,f1_macro,f1_weighted,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L1,0.892917,0.892917,0.772528,0.548983,0.855824,0.908208,accuracy,5.701250,3.042003,51.046527,5.701250,3.042003,51.046527,1,True,3
1,ExtraTreesGini_BAG_L1,0.892746,0.892746,0.776625,0.519798,0.849495,0.903921,accuracy,1.563909,1.661474,11.413453,1.563909,1.661474,11.413453,1,True,8
2,ExtraTreesEntr_BAG_L1,0.892644,0.892644,0.783967,0.518171,0.849101,0.904055,accuracy,1.546933,1.705975,11.626976,1.546933,1.705975,11.626976,1,True,9
3,CatBoost_r9_BAG_L1,0.892337,0.892337,0.766804,0.516139,0.848529,0.905283,accuracy,0.791382,0.508227,76.605883,0.791382,0.508227,76.605883,1,True,18
4,RandomForestEntr_BAG_L1,0.892029,0.892029,0.759238,0.541791,0.853890,0.905930,accuracy,1.873071,1.959654,41.223114,1.873071,1.959654,41.223114,1,True,6
5,LightGBM_r131_BAG_L1,0.890425,0.890425,0.775658,0.549881,0.854902,0.908208,accuracy,8.803161,5.043044,64.164604,8.803161,5.043044,64.164604,1,True,16
6,LightGBM_BAG_L1,0.889879,0.889879,0.780670,0.555009,0.855750,0.908364,accuracy,2.524121,0.897843,36.756478,2.524121,0.897843,36.756478,1,True,4
7,NeuralNetFastAI_r191_BAG_L1,0.888206,0.888206,0.789355,0.578995,0.860088,0.909771,accuracy,6.841512,2.475695,449.499393,6.841512,2.475695,449.499393,1,True,17
8,CatBoost_r177_BAG_L1,0.885919,0.885919,0.771681,0.553089,0.853559,0.909816,accuracy,0.360596,0.080107,77.142937,0.360596,0.080107,77.142937,1,True,14
9,NeuralNetTorch_BAG_L1,0.885748,0.885748,0.798783,0.608698,0.865214,0.909927,accuracy,1.568022,0.403425,484.339497,1.568022,0.403425,484.339497,1,True,12


In [14]:
model_name = leaderboard.iloc[0]['model']
print("Selected model name:", model_name)

Selected model name: LightGBMXT_BAG_L1


In [15]:
predictions = predictor.predict(test_data.drop(columns=['label']), model=model_name)
predictions_prob = predictor.predict_proba(test_data.drop(columns=['label']), model=model_name)

In [16]:
true_labels = test_data['label']
pred_prob = []
for i in range(len(predictions_prob)):
  pred_prob.append(predictions_prob[true_labels[i]][i])

from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
true_labels = test_data['label']
cr = classification_report(true_labels, predictions, digits=4, zero_division=0)
cm = confusion_matrix(true_labels, predictions)
TN = cm[0][0]
FN = cm[1][0]
TP = cm[1][1]
FP = cm[0][1]
# Fall out or false positive rate
FPR = FP/(FP+TN) if (FP+TN) > 0 else 0
# False negative rate
FNR = FN/(TP+FN) if (TP+FN) > 0 else 0

roc = roc_auc_score(true_labels, predictions)
roc_prob = roc_auc_score(true_labels, pred_prob)

print(cm)
print(cr)
print(f"FPR: {FPR}")
print(f"FNR: {FNR}")
print(f"ROC: {roc}")
print(f"ROC_prob: {roc_prob}")

[[  288  2948]
 [  189 25870]]
              precision    recall  f1-score   support

           0     0.6038    0.0890    0.1551      3236
           1     0.8977    0.9927    0.9428     26059

    accuracy                         0.8929     29295
   macro avg     0.7507    0.5409    0.5490     29295
weighted avg     0.8652    0.8929    0.8558     29295

FPR: 0.9110012360939431
FNR: 0.007252772554587667
ROC: 0.5408729956757345
ROC_prob: 0.9960207963947554


In [ ]:
import sklearn.metrics
from autogluon.core.metrics import make_scorer

ag_accuracy_scorer = make_scorer(name='accuracy',
                                 score_func=sklearn.metrics.accuracy_score,
                                 optimum=1,
                                 greater_is_better=True)

ag_f1_scorer = make_scorer(name='f1',
                           score_func=sklearn.metrics.f1_score,
                           optimum=1,
                           greater_is_better=True)

ag_precision_score = make_scorer(name='precision',
                                 score_func=sklearn.metrics.precision_score,
                                 optimum=1,
                                 greater_is_better=True)

ag_recall_score = make_scorer(name='recall',
                              score_func=sklearn.metrics.recall_score,
                              optimum=1,
                              greater_is_better=True)

ag_weighted_f1_score = make_scorer(name='weighted_f1',
                                   score_func=sklearn.metrics.f1_score,
                                   average='weighted',
                                   optimum=1,
                                   greater_is_better=True)

ag_macro_f1_score = make_scorer(name='macro_f1',
                                score_func=sklearn.metrics.f1_score,
                                average='macro',
                                optimum=1,
                                greater_is_better=True)

ag_weighted_precision_score = make_scorer(name='weighted_precision',
                                          score_func=sklearn.metrics.precision_score,
                                          average='weighted',
                                          optimum=1,
                                          greater_is_better=True)

ag_weighted_recall_score = make_scorer(name='weighted_recall',
                                       score_func=sklearn.metrics.recall_score,
                                       average='weighted',
                                       optimum=1,
                                       greater_is_better=True)

ag_roc_auc_score = make_scorer(name='roc_auc',
                               score_func=sklearn.metrics.roc_auc_score,
                               optimum=1,
                               greater_is_better=True,
                               needs_threshold=True)


In [ ]:
predictor_custom = TabularPredictor(label=label, eval_metric=ag_accuracy_scorer).fit(train_data, hyperparameters='toy')



No path specified. Models will be saved in: "AutogluonModels/ag-20240618_153527"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Apr 28 14:29:16 UTC 2024
CPU Count:          2
Memory Avail:       9.91 GB / 12.67 GB (78.2%)
Disk Space Avail:   169.63 GB / 201.23 GB (84.3%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : F

In [ ]:
predictor_custom.evaluate(test_data)

{'accuracy': 0.8905273937532002,
 'balanced_accuracy': 0.5189605635324728,
 'mcc': 0.1310609197882148,
 'roc_auc': 0.7442271343847429,
 'f1': 0.9418083504200613,
 'precision': 0.8932947817706182,
 'recall': 0.9958939329981964}

In [ ]:
predictor_custom.leaderboard(test_data, extra_metrics=[ag_accuracy_scorer, ag_f1_scorer, ag_macro_f1_score, ag_precision_score, ag_recall_score, ag_roc_auc_score, ag_weighted_f1_score, ag_weighted_precision_score, ag_weighted_recall_score])

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,model,score_test,accuracy,f1,macro_f1,precision,recall,roc_auc,weighted_f1,weighted_precision,...,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.890527,0.890527,0.941808,0.509996,0.893295,0.995894,0.744227,0.846410,0.856442,...,accuracy,0.190853,0.035205,7.859732,0.002481,0.000912,0.067192,2,True,5
1,NeuralNetTorch,0.889913,0.889913,0.941416,0.515328,0.893825,0.994359,0.742924,0.847282,0.852314,...,accuracy,0.174551,0.033039,7.500353,0.174551,0.033039,7.500353,1,True,4
2,CatBoost,0.889537,0.889537,0.941540,0.470770,0.889537,1.000000,0.640790,0.837535,0.791277,...,accuracy,0.013821,0.001254,0.292188,0.013821,0.001254,0.292188,1,True,2
3,LightGBM,0.889537,0.889537,0.941540,0.470770,0.889537,1.000000,0.680764,0.837535,0.791277,...,accuracy,0.014542,0.001125,0.533194,0.014542,0.001125,0.533194,1,True,1
4,XGBoost,0.889537,0.889537,0.941540,0.470770,0.889537,1.000000,0.694272,0.837535,0.791277,...,accuracy,0.030926,0.005279,0.346675,0.030926,0.005279,0.346675,1,True,3


In [ ]:
!pip install pygraphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pygraphviz (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pygraphviz
Failed to build pygraphviz
ERROR: Could not build wheels for pygraphviz, which is required to install pyproject.toml-based projects


In [ ]:
predictor_custom.plot_ensemble_model()

ImportError: Visualizing ensemble network architecture requires the `pygraphviz` library. Try `sudo apt-get install graphviz graphviz-dev` followed by `pip install pygraphviz` to install on Linux, or refer to the method docstring for detailed installation instructions for other operating systems.

In [ ]:
model_object = predictor._learner.load_model('NeuralNetTorch')

AttributeError: 'DefaultLearner' object has no attribute 'load_model'

In [ ]:
leader_model = predictor.get_model_best()

<ipython-input-14-62f26990bb27>:1: DeprecationWarning: `get_model_best` has been deprecated and will be removed in version 1.2. Please use `model_best` instead. This will raise an error in the future!
  leader_model = predictor.get_model_best()


In [ ]:
print(leader_model)

WeightedEnsemble_L2


In [ ]:
predictor.info()

{'path': 'AutogluonModels/ag-20240610_182635',
 'label': 'label',
 'random_state': 0,
 'version': '1.1.0',
 'features': ['word_count',
  'unique_word_count',
  'char_count',
  'avg_word_length',
  'ttr',
  'hapax_legomenon',
  'sentence_count',
  'avg_sentence_length',
  'avg_sentence_complexity',
  'punctuation_count',
  'noun_count',
  'stopword_count',
  'verb_count',
  'adj_count',
  'adv_count',
  'complex_sentence_count',
  'question_mark_count',
  'exclamation_mark_count',
  'flesch_reading_ease',
  'gunning_fog_index',
  'first_person_pronoun_count',
  'person_entity_count',
  'date_entity_count',
  'uniqueness_bigram',
  'uniqueness_trigram',
  'syntax_variety'],
 'feature_metadata_in': <autogluon.common.features.feature_metadata.FeatureMetadata at 0x7a565de4da50>,
 'time_fit_preprocessing': 0.21941423416137695,
 'time_fit_training': 8.248276710510254,
 'time_fit_total': 8.46769094467163,
 'time_limit': None,
 'time_train_start': 1718043995.8198974,
 'num_rows_train': 42286,
 

In [ ]:
predictor.evaluate(test_data)

{'weighted_f1': 0.8502475537678932,
 'accuracy': 0.8906639358252262,
 'balanced_accuracy': 0.5283747039083271,
 'mcc': 0.1578300381944139,
 'roc_auc': 0.7531382740819528,
 'f1': 0.9417392727868018,
 'precision': 0.8951863891002144,
 'recall': 0.9933995932307457}